In [26]:
import yaml
from json import dumps, loads
from kafka import KafkaProducer, KafkaConsumer
from fedrec.communications.messages import JobSubmitMessage

with open("../configs/dlrm_fl.yml", 'r') as cfg:
    config = yaml.load(cfg, Loader=yaml.FullLoader)

def init_kafka(config):
    producer_url = "{}:{}".format(
        config["producer_url"], config["producer_port"])
    return KafkaProducer(
        bootstrap_servers=[producer_url],
        value_serializer=lambda x: x.encode('utf-8'))

# config = config["multiprocessing"]["communications"]

producer = init_kafka(config["multiprocessing"]["communications"])
producer.send('testing1', value=JobSubmitMessage("test_run",[1,2],{},"id1","id2",None).toJSON())
# producer.send('testing1', value={"test": "test_abhinav"})

In [14]:
type(loads(JobSubmitMessage("test_run",[1,2],{},"id1","id2",None).toJSON()))

dict

In [ ]:
consumer = KafkaConsumer(bootstrap_servers=config["consumer_url"],
                auto_offset_reset='earliest',
                enable_auto_commit=True,
                group_id=config["consumer_group_id"],
                value_deserializer=lambda x: loads(x.decode('utf-8')))
                
for message in consumer:
    # message value and key are raw bytes -- decode if necessary!
    # e.g., for unicode: `message.value.decode('utf-8')`
    print ("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition,
                                          message.offset, message.key,
                                          message.value))
